# Import

In [15]:
%load_ext autoreload
%autoreload 2
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
# Prevent Keras info message; "Using TensorFlow backend."
STDERR = sys.stderr
sys.stderr = open(os.devnull, "w")
sys.stderr = STDERR
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read dataset

In [2]:
df = pd.read_csv('./data/creditcard.csv', index_col=0)
df = df.reset_index(drop=True)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# Imbalance problem
df.groupby('Class').size()

Class
0    284315
1       492
dtype: int64

# Handle Imbalance problem
Under sampling technique

In [4]:
class_0_df = df[df['Class'] == 0].sample(n=1008, random_state=123)
class_1_df = df[df['Class'] == 1]

balanced_df = class_0_df.append(class_1_df, ignore_index=True)
balanced_df = balanced_df.sample(frac=1, random_state=123).reset_index(drop=True)
balanced_df.groupby('Class').size()

Class
0    1008
1     492
dtype: int64

# Train-Test split

In [5]:
def train_size(num):
    x_train = balanced_df[balanced_df.columns.difference(['Class'])][:num]
    y_train = balanced_df['Class'][:num]
    return x_train.values, y_train.values

x_train, y_train = train_size(1200)
x_test, y_test = train_size(1500)
x_test = x_test[1200:1500]; y_test = y_test[1200:1500]

In [6]:
x_train.shape

(1200, 29)

# Tune Parameter

## - Batch Size and Number of Epochs

In [16]:
def create_model():
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(12, activation='relu', input_shape=(x_train.shape[1],)))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 30]
epochs = [10, 30, 50]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.955833 using {'batch_size': 30, 'epochs': 50}
0.954167 (0.006972) with: {'batch_size': 10, 'epochs': 10}
0.952500 (0.008975) with: {'batch_size': 10, 'epochs': 30}
0.954167 (0.007454) with: {'batch_size': 10, 'epochs': 50}
0.948333 (0.009718) with: {'batch_size': 20, 'epochs': 10}
0.951667 (0.013591) with: {'batch_size': 20, 'epochs': 30}
0.955000 (0.006667) with: {'batch_size': 20, 'epochs': 50}
0.920000 (0.026273) with: {'batch_size': 30, 'epochs': 10}
0.952500 (0.011055) with: {'batch_size': 30, 'epochs': 30}
0.955833 (0.013070) with: {'batch_size': 30, 'epochs': 50}


## - Training Optimization Algorithm

In [8]:
def create_model(optimizer='adam'):
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(12, activation='relu', input_shape=(x_train.shape[1],)))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, epochs=10, batch_size=10)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.955000 using {'optimizer': 'Nadam'}
0.886667 (0.039739) with: {'optimizer': 'SGD'}
0.951667 (0.013591) with: {'optimizer': 'RMSprop'}
0.952500 (0.008580) with: {'optimizer': 'Adagrad'}
0.949167 (0.012191) with: {'optimizer': 'Adadelta'}
0.953333 (0.011304) with: {'optimizer': 'Adam'}
0.949167 (0.006124) with: {'optimizer': 'Adamax'}
0.955000 (0.010992) with: {'optimizer': 'Nadam'}


## - Network Weight Initialization

In [9]:
def create_model(init_mode='uniform'):
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(12, kernel_initializer=init_mode, activation='relu', input_shape=(x_train.shape[1],)))
    # Add an output layer 
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, epochs=10, batch_size=10)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.956667 using {'init_mode': 'lecun_uniform'}
0.950833 (0.013281) with: {'init_mode': 'uniform'}
0.956667 (0.010069) with: {'init_mode': 'lecun_uniform'}
0.954167 (0.011487) with: {'init_mode': 'normal'}
0.670833 (0.023422) with: {'init_mode': 'zero'}
0.954167 (0.014672) with: {'init_mode': 'glorot_normal'}
0.935833 (0.025631) with: {'init_mode': 'glorot_uniform'}
0.952500 (0.008165) with: {'init_mode': 'he_normal'}
0.949167 (0.015456) with: {'init_mode': 'he_uniform'}


## - Neuron Activation Function

In [10]:
def create_model(activation='relu'):
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(12, kernel_initializer='lecun_uniform', activation=activation, input_shape=(x_train.shape[1],)))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, epochs=10, batch_size=10)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.955833 using {'activation': 'softplus'}
0.947500 (0.010737) with: {'activation': 'softmax'}
0.955833 (0.013070) with: {'activation': 'softplus'}
0.953333 (0.012191) with: {'activation': 'softsign'}
0.935000 (0.047478) with: {'activation': 'relu'}
0.947500 (0.011961) with: {'activation': 'tanh'}
0.951667 (0.013070) with: {'activation': 'sigmoid'}
0.952500 (0.011667) with: {'activation': 'hard_sigmoid'}
0.955000 (0.011607) with: {'activation': 'linear'}


## - Dropout Regularization

In [11]:
def create_model(dropout_rate=0.0, weight_constraint=0):
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(12, kernel_initializer='lecun_uniform', activation='softplus', input_shape=(x_train.shape[1],), kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, epochs=10, batch_size=10)
# define the grid search parameters
weight_constraint = [1, 2, 3]
dropout_rate = [0.0, 0.1, 0.2, 0.4]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.955833 using {'dropout_rate': 0.0, 'weight_constraint': 2}
0.948333 (0.015046) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.955833 (0.010737) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.951667 (0.015943) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.949167 (0.013017) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.949167 (0.010341) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.945833 (0.016457) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.952500 (0.008165) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.945000 (0.015230) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.948333 (0.014577) with: {'dropout_rate': 0.2, 'weight_constraint': 3}
0.950833 (0.012748) with: {'dropout_rate': 0.4, 'weight_constraint': 1}
0.953333 (0.009647) with: {'dropout_rate': 0.4, 'weight_constraint': 2}
0.953333 (0.011304) with: {'dropout_rate': 0.4, 'weight_constraint': 3}


## - Number of Neurons in the Hidden Layer

In [12]:
def create_model(neurons=1):
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(neurons, kernel_initializer='lecun_uniform', activation='softplus', input_shape=(x_train.shape[1],), kernel_constraint=maxnorm(2)))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0, epochs=10, batch_size=10)
# define the grid search parameters
neurons = [5, 8, 11, 12, 13, 14, 28, 30, 32]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0)
grid_result = grid.fit(x_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.958333 using {'neurons': 32}
0.951667 (0.015046) with: {'neurons': 5}
0.953333 (0.012748) with: {'neurons': 8}
0.947500 (0.016791) with: {'neurons': 11}
0.934167 (0.041783) with: {'neurons': 12}
0.944167 (0.014814) with: {'neurons': 13}
0.926667 (0.040706) with: {'neurons': 14}
0.946667 (0.017361) with: {'neurons': 28}
0.939167 (0.034319) with: {'neurons': 30}
0.958333 (0.004564) with: {'neurons': 32}


# Define and train model

In [13]:
def create_model():
    # Initialize the constructor
    model = Sequential()
    # Add an input layer 
    model.add(Dense(32, kernel_initializer='lecun_uniform', activation='softplus', input_shape=(x_train.shape[1],), kernel_constraint=maxnorm(2)))
    # Add an output layer 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='Nadam',
                  metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
keras_model = create_model()

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

keras_model.fit(x_train, y_train, validation_split=0.1, callbacks=[early_stopping_callback], batch_size=10,
               epochs=10, shuffle=True)

Train on 1080 samples, validate on 120 samples
Epoch 1/10
1080/1080 [==============================] - 9s 8ms/step - loss: 1.0750 - acc: 0.7583 - val_loss: 0.8292 - val_acc: 0.9000
Epoch 2/10
1080/1080 [==============================] - 1s 491us/step - loss: 0.3275 - acc: 0.9352 - val_loss: 0.7418 - val_acc: 0.9083
Epoch 3/10
1080/1080 [==============================] - 1s 488us/step - loss: 0.1840 - acc: 0.9546 - val_loss: 0.3463 - val_acc: 0.9333
Epoch 4/10
1080/1080 [==============================] - 1s 482us/step - loss: 0.3606 - acc: 0.9435 - val_loss: 0.2861 - val_acc: 0.9333
Epoch 5/10
1080/1080 [==============================] - 1s 483us/step - loss: 0.2316 - acc: 0.9528 - val_loss: 0.4212 - val_acc: 0.9333


# Evaluate model

In [14]:
# evaluate the model
scores = keras_model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (keras_model.metrics_names[1], scores[1]*100))

acc: 94.00%
